# `__dict__`, `vars()`, `dir()`, `getattr()`


Поле `__dict__` содержит пространство имен класса/объекта. В `__dict__` *класса* содержатся дополнительные поля, которые не были явно определены при создании класса (например, поле `__module__`). Эти поля наследуются от суперкласа.

Встроенная функция `vars()` также возвращает пространство имен переданного в него аргумента. Фактически, `vars(obj)` возвращает `obj.__dict__`, если у объекта `obj` есть поле `__dict__`. 

In [20]:
Rectangle.__dict__              # набор атрибутов класса

mappingproxy({'__module__': '__main__',
              '__doc__': 'The class for rectangle objects',
              '_Rectangle__MIN_VALUE': 13,
              'MAX_VALUE': 17,
              'height': 50,
              'setValues': <classmethod(<function Rectangle.setValues at 0x0000028696F51260>)>,
              'validate': <classmethod(<function Rectangle.validate at 0x0000028696F50EA0>)>,
              '__init__': <function __main__.Rectangle.__init__(self, width, height) -> None>,
              'get_area': <staticmethod(<function Rectangle.get_area at 0x00000286970A8860>)>,
              '__dict__': <attribute '__dict__' of 'Rectangle' objects>,
              '__weakref__': <attribute '__weakref__' of 'Rectangle' objects>})

В поле `__dict__` *объекта* содержатся только поля, относящиеся непосредственно к объекту.

In [21]:
r.__dict__

{'_Rectangle__width': 3, 'height': 4, 'area': 12}

Заметим, что `__dict__` содержит также и приватные имена с префиксом `_Rectangle`.

Значения по ключам в `__dict__` ссылаются на соответствующие поля и методы, поэтому к ним можно получить доступ и (даже изменить) через `__dict__` (даже к приватным):

In [22]:
r.__dict__["_Rectangle__width"] = 5

In [23]:
r.__dict__.keys()

dict_keys(['_Rectangle__width', 'height', 'area'])

Встроенна функция `dir()` возвращает список, содержащий имена полей и методов, доступных через имя объекта или класса.

In [24]:
print( dir(Rectangle) )
print( dir(r) )

['MAX_VALUE', '_Rectangle__MIN_VALUE', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'get_area', 'height', 'setValues', 'validate']
['MAX_VALUE', '_Rectangle__MIN_VALUE', '_Rectangle__width', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'area', 'get_area', 'height', 'setValues', 'validate']


В этот список входят также добавочные поля и методы реализации класса: все классы наследуют крупный набор имен методов для перегрузки операций. Фактически обычно вы захотите отфильтровать большинство имен из результатов `dir()`, т.к. они относятся к внутренним деталям реализации, которые в нормальной ситуации отображать нежелательно:

In [25]:
[name for name in dir(r) if not name.startswith('__')]

['MAX_VALUE',
 '_Rectangle__MIN_VALUE',
 '_Rectangle__width',
 'area',
 'get_area',
 'height',
 'setValues',
 'validate']

Функция `getattr()` возвращает значение атрибута объекта, если атрибут определен в пространстве имен объекта или класса объекта и является публичным, в противном случае возвратит значение по умолчанию, задаваемое третьим аргументом.

In [26]:
print(f"{ getattr(r, 'height', False) = }")
print(f"{ getattr(r, '__width', False) = }")
print(f"{ getattr(r, 'validate', False) = }")

 getattr(r, 'height', False) = 4
 getattr(r, '__width', False) = False
 getattr(r, 'validate', False) = <bound method Rectangle.validate of <class '__main__.Rectangle'>>


# Класс-последовательность

Благодаря определенным магическим методам можно придать объектам класса свойства, характерные для последовательностей, таких как `list` или `tuple`.

Для того, чтобы можно было обращаться к элементам объекта через квадратные скобки `[]`, следует переопределить методы `__getitem__()`, `__setitem__()` и `__delitem__()`.

Последовательности характерны тем, что являются итерируемыми. Функция `iter()` может создать итератор на основе метода `__getitem__()`, однако в первую очередь функция пробует вызвать метод `__iter__()`, ожидая получить от в результате вызова итератор. Для этого в методе можно определить поле счетчика, и вернуть сам объект (`self`). При этом, чтоб объект был итератором, необходимо также, чтобы был определен метод `__next__()`, который будет вызываться функцией `next()`.

In [51]:
class Alphabet:
    def __init__(self, register="lowercase"):
        if register == "lowercase":
            self.__container = [chr(x) for x in range(97, 123)]
        elif register == "uppercase":
            self.__container = [chr(x) for x in range(65, 91)]
        else:
            raise ValueError("argument register must be 'lowercase' or 'uppercase'")

    def __getitem__(self, index):
        return self.__container[index]

    def __iter__(self):
        self.counter = 0
        return self

    def __next__(self):
        if self.counter < len(self.__container):
            value = self.__container[self.counter]
            self.counter += 1
            return value
        else:
            raise StopIteration

In [48]:
a = Alphabet("uppercase")
a[9]

'J'

In [53]:
class Alphabet:
    def __init__(self, register="lowercase"):
        if register == "lowercase":
            self.__container = [chr(x) for x in range(97, 123)]
        elif register == "uppercase":
            self.__container = [chr(x) for x in range(65, 91)]
        else:
            raise ValueError("argument register must be 'lowercase' or 'uppercase'")

    def __getitem__(self, index):
        return self.__container[index]

    def __iter__(self):
        # Генератор — каждый вызов создаёт новый объект
        for item in self.__container:
            yield item

In [54]:
a = Alphabet()

for x in a:
    for y in a:  # внутренний цикл сбрасывает counter!
        print(f"{x}{y}", end=" ")
    print()

aa ab ac ad ae af ag ah ai aj ak al am an ao ap aq ar as at au av aw ax ay az 
ba bb bc bd be bf bg bh bi bj bk bl bm bn bo bp bq br bs bt bu bv bw bx by bz 
ca cb cc cd ce cf cg ch ci cj ck cl cm cn co cp cq cr cs ct cu cv cw cx cy cz 
da db dc dd de df dg dh di dj dk dl dm dn do dp dq dr ds dt du dv dw dx dy dz 
ea eb ec ed ee ef eg eh ei ej ek el em en eo ep eq er es et eu ev ew ex ey ez 
fa fb fc fd fe ff fg fh fi fj fk fl fm fn fo fp fq fr fs ft fu fv fw fx fy fz 
ga gb gc gd ge gf gg gh gi gj gk gl gm gn go gp gq gr gs gt gu gv gw gx gy gz 
ha hb hc hd he hf hg hh hi hj hk hl hm hn ho hp hq hr hs ht hu hv hw hx hy hz 
ia ib ic id ie if ig ih ii ij ik il im in io ip iq ir is it iu iv iw ix iy iz 
ja jb jc jd je jf jg jh ji jj jk jl jm jn jo jp jq jr js jt ju jv jw jx jy jz 
ka kb kc kd ke kf kg kh ki kj kk kl km kn ko kp kq kr ks kt ku kv kw kx ky kz 
la lb lc ld le lf lg lh li lj lk ll lm ln lo lp lq lr ls lt lu lv lw lx ly lz 
ma mb mc md me mf mg mh mi mj mk ml mm mn mo mp mq m

In [49]:
for x in a:
    print(x, end=' ')

A B C D E F G H I J K L M N O P Q R S T U V W X Y Z 

> Если при получении доступа к элементу в квадратных скобках указано число (`a[9]`), то метод `__getitem__()` получает в качестве аргумента указанное число. Если же задать в квадратных скобках срез (например `a[:4]` или `a[1:10:2]`), то `__getitem__()` получит в качестве аргумента объект класса `slice`. Получить указанные значения можно в полях `start`, `stop` и `step`.

In [50]:
s = slice(1, 20, 3)
s.start, s.stop, s.step

(1, 20, 3)